<a href="https://colab.research.google.com/github/pullz6/Research_inspired_Self-Explaining-Neural-Networks-for-Business-Process-Monitoring/blob/main/Data_preprocessing_Model_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.7 MB/s eta 0:00:00


In [3]:
!pip install -q kaggle

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from faker import Faker
import pyspark
import os
from google.colab import drive
from kaggle.api.kaggle_api_extended import KaggleApi
import json

In [7]:
#Read creds
drive.mount('/content/drive')

#loading all the required datasets
creds_path = ('/content/drive/MyDrive/Projects/Creds/kaggle.json')

Mounted at /content/drive


In [8]:
with open(creds_path, 'r') as f:
    creds = json.load(f)

In [9]:
!mkdir -p ~/.kaggle

In [10]:
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(creds, f)

In [11]:
# Set permissions
!chmod 600 ~/.kaggle/kaggle.json

# Verify setup
!kaggle datasets list -s "ieee fraud"

# STEP 2: DOWNLOAD DATASET
# Download the dataset (this is a known mirror of the original competition data)
!kaggle datasets download -d kartik2112/fraud-detection

# Unzip the files (creates a 'fraud-detection' folder)
!unzip fraud-detection.zip -d fraud-detection

ref                                                        title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------  -------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
mlg-ulb/creditcardfraud                                    Credit Card Fraud Detection                          69155672  2018-03-23 01:17:27.913000         897888      12222  0.85294116       
muhakabartay/yourallmodelsdata                             IEEE-CIS Fraud Detection Models Data                 29768007  2019-09-18 07:57:04.473000            682         17  1.0              
whenamancodes/fraud-detection                              Fraud Detection                                      69155672  2022-09-12 11:54:40.550000          10576        117  1.0              
kyakovlev/ieee-submissions-and

In [12]:
# List downloaded files
!ls -la fraud-detection/

total 489852
drwxr-xr-x 2 root root      4096 Jun  9 13:55 .
drwxr-xr-x 1 root root      4096 Jun  9 13:55 ..
-rw-r--r-- 1 root root 150354339 Aug  5  2020 fraudTest.csv
-rw-r--r-- 1 root root 351238196 Aug  5  2020 fraudTrain.csv


In [13]:
from pyspark.sql import SparkSession

In [14]:
spark = SparkSession.builder.appName("IEEE_Fraud_Detection").config("spark.driver.memory", "8g").config("spark.executor.memory", "8g").getOrCreate()

In [15]:
# Load the training and testing data
test = spark.read.csv(
    f"fraud-detection/fraudTest.csv",
    header=True,
    inferSchema=True
)

train = spark.read.csv(
    f"fraud-detection/fraudTrain.csv",
    header=True,
    inferSchema=True
)

In [16]:
# DataFrame approach
print(f"Training transactions: {train.count():,} rows")
train.printSchema()

# Show some statistics
train.describe().show()

Training transactions: 1,296,675 rows
root
 |-- _c0: integer (nullable = true)
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)

+-------+-----------------+--------------------+-------------------+

In [17]:
test.describe().show()

+-------+-----------------+--------------------+-------------------+-------------+-----------------+-------+------+------+--------------------+-------+------+------------------+-----------------+------------------+------------------+------------------+--------------------+--------------------+-----------------+------------------+--------------------+
|summary|              _c0|              cc_num|           merchant|     category|              amt|  first|  last|gender|              street|   city| state|               zip|              lat|              long|          city_pop|               job|           trans_num|           unix_time|        merch_lat|        merch_long|            is_fraud|
+-------+-----------------+--------------------+-------------------+-------------+-----------------+-------+------+------+--------------------+-------+------+------------------+-----------------+------------------+------------------+------------------+--------------------+--------------------+